## Documentation of Point Transformer modifications

#### Gazebo Data Structure
05.02.2025 
- 9988 clouds w/poses
- gazebo_pc_record_full_12_42_labeled_1024
    - clouds/
    - poses/

    Up/Downsampling used:
    - clouds with < 1024: random duplication with slight gaussian noise
    - clouds with > 1024: FPS (takes a long time)
    - not removing any clouds

#### 1. train_pose.py
Dataloader: SimNet or ScanNet

#### 2. pointtransformer_pose.py
```self.radius = 1.5```
```input_dim```: change based on number of one-hot encoded classes used

In [ ]:
# uniform=False means our .ply files are already downsampled to 1024 points


#### 3. Translation-only prediction: _t.py
- use train_pose_t.py, test_pose_t.py, and pointtransformer_pose_t.py

Model Training+Inference Process: 
0. ```$ conda activate point-transformer```
1. train_pose.py
    - pointtransformer_pose.py: contains PoseLoss()
        - quaternion_to_rotation_matrix() assumes wxyz format as of 12.02.25
2. test_pose.py
    - manually specify best model saved during training to use for inference
    - writes results as .json in XYZW format
3. overlay_pose_estimate.py (MAC)
    - reads from json and for each cloud file we used in training set (which the .json stores the pose prediction for), overlays the cloud on the stl model for visual evaluation.

#### Evaluation+Improvements: 12.02.25
- Rotate cloud by prediction, then see how much they overlap (using some metric like average distance of model points to transformed cloud (AD/ADS))
- Need more input (number of scans, labels, etc)?
- Use the ground truth model somehow? i.e. dense ship cloud?
- Experiment with MSG radius
- Data augmentation beyond sensor pose variation
- Examine what the Point Transformer portion learns. How is it representing scans?
- Study how point distribution affects prediction (stern deck is denser than port, for eg.)
- Try maybe an ICP approach for refinement?
    - For this, first plot the input cloud in the space that the model reads it in. The idea is we want to see whether the raw clouds are "perturbed" enough by default, (since they're in the sensor frame), or whether the model actually does learn most of the rotation in the SE3 space.
- Big IDEA for now: review how the input scans are represneted to the model, how+what does it learn about the scan that lets it be roughly transformed "correctly" to the ship in pyvista.
- Try this with the 4.0 val error model. How much do more epochs improve the predictions?

#### Initial Observations:
- Is translation is struggling more than rotation? (either too low or too high)
- 008054.txt is quite off in the yaw direction

#### 13.02:
- [ ] Predict position first
    - Don't forget about unit sphere scaling + centroid and how that affects it!
    - read more about decoupling rot and trans. does this allow the model to learn each better? i.e. would I try to learn rotation around origin? This again fits into the unit sphere situation i have right now.
- [ ] visualize top K 
- [ ] quickly NEXT, train by concatenating scale, centroid?
- [ ] try quaternion loss
- [ ] NEXT, try training without unit-sphere scaling (just centroid)
    - might need to experiment with adjusting radii if not unit-sphere, or concatenating global scale? think...
- [ ] try with trained data
- [ ] use ICP (geometric registration problem)? after transformer gets approximate pose
    - "In order to improve the registration perfor-
mance, features on point clouds are also introduced for
matching. "
- [ ] loss function predicting angle-axis, but does this cause issue with conversion?
    - try different loss function (see GDR-Net), other parameterization besides angle axis
- [ ] then try keypoint prediction
    - plot the predicted keypoints after unit sphere scaling applied to them to ensure 

Question: should validation be used durign training and fed back? Or just printed?


QUATERNION FORMAT: WXYZ or XYZW: 12.02.25
- gt stored as WXYZ quaterion
- model learns angle-axis
    - within loss function, its converted to rotation matrix to compare with ground truth (also converted to matrix)
- NOTE: during inference (test_pose.py), we write pred_quat as XYZW to results.json!!
    -  now writes ground truth gt_quat to be XYZW to results.json
- on Mac, in overlay_pose_estimate.py, reads from results.json which has all XYZW format.

<!-- TODO: ground truth should be as XYZW too -->